### Propagación hacia adelante (forward propagation)

Vamos a implementar una red neuronal con capacidad de hacer predicciones mediante la propagación hacia adelante

### Cargamos los datos

Vamos a usar el dataset del cancer de mama (Breast Cancer Dataset)

In [1]:
import numpy as np

In [2]:
from sklearn.datasets import load_breast_cancer

In [3]:
data = load_breast_cancer()

In [4]:
X, y = data.data, data.target

Para poder usar la red neuronal que hemos visto con 4 unidades en la capa de entrada, usaremos solo las 4 primeras variables independientes

In [5]:
X = data.data[:,:4]

In [6]:
X.shape

(569, 4)

In [7]:
y[:20]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])

In [8]:
X[:10]

array([[  17.99,   10.38,  122.8 , 1001.  ],
       [  20.57,   17.77,  132.9 , 1326.  ],
       [  19.69,   21.25,  130.  , 1203.  ],
       [  11.42,   20.38,   77.58,  386.1 ],
       [  20.29,   14.34,  135.1 , 1297.  ],
       [  12.45,   15.7 ,   82.57,  477.1 ],
       [  18.25,   19.98,  119.6 , 1040.  ],
       [  13.71,   20.83,   90.2 ,  577.9 ],
       [  13.  ,   21.82,   87.5 ,  519.8 ],
       [  12.46,   24.04,   83.97,  475.9 ]])

Para entrenar modelos de deep learning es importante que las variables estén en la misma escala, por ello vamos a estandarizar las variables independientes

In [9]:
from sklearn.preprocessing import StandardScaler

In [10]:
x_estandardizador = StandardScaler()
X_std = x_estandardizador.fit_transform(X)

En primer lugar, tenemos que definir una capa, creamos la clase `Layer` que tiene una dimension de entrada, una de salida, y una función de activación

Inicialmente sus pesos se generan al azar.

In [11]:
class Layer:
    def __init__(self, dim_input, dim_output, fn_activacion, nombre):
        self.dim_input = dim_input
        self.dim_output = dim_output
        self.generar_pesos((dim_output, dim_input))
        self.generar_bias(dim_output)
        self.fn_activacion = fn_activacion
        self.nombre = nombre
        
    def __repr__(self):
        return """
        Capa {}. tamaño input: {}. tamaño output: {}.
        pesos: {}
        bias: {}
        """.format(
        self.nombre, self.dim_input, self.dim_output,
        self.w, self.b)
    
    def generar_pesos(self, dimensiones):
        self.w = np.random.random(dimensiones)
        
    def generar_bias(self, dim_output):
        self.b = np.random.random((dim_output,))
    
    def activar(self, x):
        return self.fn_activacion(self.w @ x + self.b)

Para hacer propagación hacia adelante necesitamos la función de activación,  en este ejemplo voy a usar la función sigmoide como activación de la capa oculta

In [12]:
def fn_sigmoide(x):
    return 1/(1+np.exp(-x))

In [13]:
n_input = 4
n_oculta = 5
n_output = 1

In [14]:
capa_oculta = Layer(n_input, n_oculta, fn_sigmoide, "oculta")

capa_salida = Layer(n_oculta, n_output, fn_sigmoide, "salida")

In [15]:
print(capa_oculta)


        Capa oculta. tamaño input: 4. tamaño output: 5.
        pesos: [[0.69797644 0.08610869 0.00952704 0.68134878]
 [0.10080798 0.14013767 0.80691453 0.26764593]
 [0.97526905 0.75329126 0.65565684 0.18434846]
 [0.19676045 0.41162016 0.11416453 0.83939117]
 [0.73601958 0.02570742 0.09535545 0.6205065 ]]
        bias: [0.68676302 0.45467548 0.6809074  0.04339686 0.4732156 ]
        


Ahora podemos crear una red neuronal, que básicamente tiene una lista con las capas que tiene y el método para la propagación hacia adelante

In [16]:
class RedNeuronal:
    def __init__(self):
        self.layers = []
        
    def add_layer(self, layer):
        self.layers.append(layer)
        
    def forward(self, x):
        print("""
        input {}
        """.format(x))
        for layer in self.layers:
            x = layer.activar(x)
            print(layer)
            print("""
            output: {}
            """.format(x))
        return x

In [17]:
red = RedNeuronal()

red.add_layer(capa_oculta)
red.add_layer(capa_salida)

In [18]:
indice_aleatorio = np.random.permutation(X.shape[0])

x0 = X_std[indice_aleatorio[0]]
y0 = y[indice_aleatorio[0]]
print(x0, y0)

[-1.57068136 -0.16048584 -1.56024497 -1.23345631] 1


In [19]:
red.forward(x0)


        input [-1.57068136 -0.16048584 -1.56024497 -1.23345631]
        

        Capa oculta. tamaño input: 4. tamaño output: 5.
        pesos: [[0.69797644 0.08610869 0.00952704 0.68134878]
 [0.10080798 0.14013767 0.80691453 0.26764593]
 [0.97526905 0.75329126 0.65565684 0.18434846]
 [0.19676045 0.41162016 0.11416453 0.83939117]
 [0.73601958 0.02570742 0.09535545 0.6205065 ]]
        bias: [0.68676302 0.45467548 0.6809074  0.04339686 0.4732156 ]
        

            output: [0.21778296 0.21160703 0.09777516 0.17578246 0.16782684]
            

        Capa salida. tamaño input: 5. tamaño output: 1.
        pesos: [[0.03836902 0.91605681 0.17978807 0.06883539 0.05385343]]
        bias: [0.27003112]
        

            output: [0.62502873]
            


array([0.62502873])

In [20]:
y0

1